In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.insert(0, '../../')

from bayesbridge import BayesBridge

## Specify mean and sd in log10 scale and find the matching prior.

In [ ]:
bridge_exponent = 1 / 8
log10_mean = - 4.
log10_sd = 1.
gscale_parametrization = ['raw', 'coefficient'][1]

In [ ]:
# Dummy data just to initialize BayesBridge
y = np.random.randn(10)
X = np.random.randn(10, 2)
bridge = BayesBridge(y, X, global_scale_parametrization=gscale_parametrization)
bridge.set_global_scale_prior(log10_mean, log10_sd, bridge_exponent)
prior_param = bridge.prior_param['gscale_neg_power']

## Check that the prior indeed has the specified mean and sd.

In [ ]:
shape = prior_param['shape']
scale = prior_param['rate'] ** -1

In [ ]:
n_sample = 10 ** 6
samples = np.random.gamma(shape, scale, size=n_sample) ** (- 1 / bridge_exponent)
if gscale_parametrization == 'coefficient':
    samples *= bridge.compute_power_exp_ave_magnitude(bridge_exponent, 1.)
log10_gscale_samples = np.log10(samples)

mean_est = np.mean(log10_gscale_samples)
sd_est = np.std(log10_gscale_samples)

rtol = .01
mean_is_close = abs((mean_est - log10_mean) / log10_mean) < rtol
sd_is_close = abs((sd_est - log10_sd) / log10_sd) < rtol
if mean_is_close and sd_is_close:
    print("Monte Carlo estimates agree with theoretical values.")
else:
    print("Warning! Monte Carlo estimates do NOT agree with theoretical values.")

In [ ]:
plt.figure(figsize=(7, 5))
plt.rcParams['font.size'] = 18

plt.hist(
    log10_gscale_samples, bins=51, density=True,
    label='prior dist.'
)
plt.axvline(
    mean_est, linestyle='--', color='tab:orange',
    label='mean'
)
plt.axvline(
    mean_est + 2 * sd_est, linestyle='--', color='tab:olive',
    label=r'mean $\pm$ 2 std'
)
plt.axvline(
    mean_est - 2 * sd_est, linestyle='--', color='tab:olive'
)
plt.xlabel(r'$\log(\tau)$')
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()